# User features

## Import packages

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import random
random.seed(32)
import gensim
from gensim import corpora, models
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import itertools

## Import data

In [2]:
# authors = pd.read_csv('/home/sophia/ma_py/author_profiles.csv')
global df
df = pd.read_pickle("/home/sophia/ma_py/Big5-NLP/old_files/preprocessed_author.pkl")
# df = pre_df.set_index(['index', 'author'])
# df.head(10)
df.head(n=20)

,index,author,complete_body,doc_body,probody,tokens,senttokens,agreeableness,openness,conscientiousness,...,fantasyfootball,nostalgia,watchpeopledie,homebrewing,nexus6p,canadapolitics,rottweiler,ipad,samplesize,starwarsbattlefront
0,0,-BigSexy-,Oooh i see,[Oooh i see],[oooh see],"[[oooh, see]]",[[Oooh i see]],39.0,92.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,1,-BlitzN9ne,**Quality** material right here,[**Quality** material right here],[quality material right],"[[quality, material, right]]",[[**Quality** material right here]],50.0,85.0,15.0,...,0,0,0,0,0,0,0,0,0,0
2,2,-CrestiaBell,A slidewhistle or a meow-meow board That's bec...,"[A slidewhistle or a meow-meow board, That's b...","[slidewhistle meow meow board, watch cartoon s...","[[slidewhistle, meow, meow, board], [watch, ca...","[[A slidewhistle or a meow-meow board], [That'...",50.0,85.0,50.0,...,0,0,0,0,0,0,0,0,0,0
3,3,-tactical-throw-away,Sorry for your feelings. Kek &lt;------- This ...,"[Sorry for your feelings., Kek &lt;------- Thi...","[sorry feelings, kek lt onekek kek kek kek, no...","[[sorry, feelings], [kek, lt, onekek, kek, kek...","[[Sorry for your feelings.], [Kek &lt;------- ...",2.0,92.0,31.0,...,0,0,0,0,0,0,0,0,0,0
4,4,137288,Carly's so glad to get your .0000003 cents Exc...,"[Carly's so glad to get your .0000003 cents, E...","[carly glad get three cents, except uk debuted...","[[carly, glad, get, three, cents], [except, uk...","[[Carly's so glad to get your .0000003 cents],...",10.0,87.0,49.0,...,0,0,0,0,0,0,0,0,0,0
5,5,1fastman1,The weak should fear the strong The rock could...,"[The weak should fear the strong, The rock cou...","[weak fear strong, rock couldnt save, recently...","[[weak, fear, strong], [rock, couldnt, save], ...","[[The weak should fear the strong], [The rock ...",70.0,42.0,28.0,...,0,0,0,0,0,0,0,0,0,0
6,6,3062226abcd,DOWNVOTE SO THE DEVS DONT FIND OUT...jk lol co...,[DOWNVOTE SO THE DEVS DONT FIND OUT...jk lol c...,[downvote devs dont find jk lol cool trick btw],"[[downvote, devs, dont, find, jk, lol, cool, t...",[[DOWNVOTE SO THE DEVS DONT FIND OUT...jk lol ...,47.0,50.0,57.0,...,0,0,0,0,0,0,0,0,0,0
7,7,5426742,Such acrobats.,[Such acrobats.],[acrobats],[[acrobats]],[[Such acrobats.]],28.0,88.0,63.0,...,0,0,0,0,0,0,0,0,0,0
8,8,ACE_C0ND0R,2015 Battlefront doesn't even deserve to be on...,[2015 Battlefront doesn't even deserve to be o...,"[battlefront not even deserve list, republican...","[[battlefront, not, even, deserve, list], [rep...",[[2015 Battlefront doesn't even deserve to be ...,10.0,80.0,74.0,...,0,0,0,0,0,0,0,0,0,0
9,9,Abcmsaj,Shame it looks like a deformed squirrel now th...,[Shame it looks like a deformed squirrel now t...,[shame looks like deformed squirrel thanks sco...,"[[shame, looks, like, deformed, squirrel, than...",[[Shame it looks like a deformed squirrel now ...,81.0,61.0,60.0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
n = len(df.columns.values.tolist())

headers = n*['x']
columns = df.columns.values
print(len(headers))
print(len(columns))
arrays = [headers] + [columns]
df.columns=pd.MultiIndex.from_arrays(arrays)
df

477
477


x                        \
    index                author   
0       0             -BigSexy-   
1       1            -BlitzN9ne   
2       2          -CrestiaBell   
3       3  -tactical-throw-away   
4       4                137288   
..    ...                   ...   
417   424           xanthraxoid   
418   425             xenomouse   
419   426              xeroctr3   
420   427               xzack18   
421   428           zugzwang_03   

                                                        \
                                         complete_body   
0                                           Oooh i see   
1                      **Quality** material right here   
2    A slidewhistle or a meow-meow board That's bec...   
3    Sorry for your feelings. Kek &lt;------- This ...   
4    Carly's so glad to get your .0000003 cents Exc...   
..                                                 ...   
417  I'd really like this video to include some inf...   
418  You're a guy, aren't you? I can definitely see...   
419  man even the thought of it makes me depressed....   
420                      Not all of us are out to kill   
421  Institutions should accommodate religious or s...   

                                                        \
                                              doc_body   
0                                         [Oooh i see]   
1                    [**Quality** material right here]   
2    [A slidewhistle or a meow-meow board, That's b...   
3    [Sorry for your feelings., Kek &lt;------- Thi...   
4    [Carly's so glad to get your .0000003 cents, E...   
..                                                 ...   
417  [I'd really like this video to include some in...   
418  [You're a guy, aren't you? I can definitely se...   
419  [man even the thought of it makes me depressed...   
420                    [Not all of us are out to kill]   
421  [Institutions should accommodate religious or ...   

                                                        \
                                               probody   
0                                           [oooh see]   
1                             [quality material right]   
2    [slidewhistle meow meow board, watch cartoon s...   
3    [sorry feelings, kek lt onekek kek kek kek, no...   
4    [carly glad get three cents, except uk debuted...   
..                                                 ...   
417  [would really like video include information c...   
418  [guy not definitely see would make boy scene f...   
419  [man even thought makes depressed loving someo...   
420                                      [not us kill]   
421  [institutions accommodate religious serious me...   

                                                        \
                                                tokens   
0                                        [[oooh, see]]   
1                         [[quality, material, right]]   
2    [[slidewhistle, meow, meow, board], [watch, ca...   
3    [[sorry, feelings], [kek, lt, onekek, kek, kek...   
4    [[carly, glad, get, three, cents], [except, uk...   
..                                                 ...   
417  [[would, really, like, video, include, informa...   
418  [[guy, not, definitely, see, would, make, boy,...   
419  [[man, even, thought, makes, depressed, loving...   
420                                  [[not, us, kill]]   
421  [[institutions, accommodate, religious, seriou...   

                                                                               \
                                            senttokens agreeableness openness   
0                                       [[Oooh i see]]          39.0     92.0   
1                  [[**Quality** material right here]]          50.0     85.0   
2    [[A slidewhistle or a meow-meow board], [That'...          50.0     85.0   
3    [[Sorry for your feelings.], [Kek &lt;------- ...           2.0     92.0   
4    [[Carly's so glad to get your .0000003 ce

## LDA

In [4]:
# User features

# Preprocessing for LDA
def preprocess_lda():
    global df
    neglst = ["no", "not", "none", "nobody", "nothing", "neither", "nowhere", "never", "nay"]
    inputlst = []
    for row in tqdm(df['x', 'tokens']):
        rowlst = []
        for comment in row:
            rowlst.append([word for word in comment if (word not in neglst)])
        inputlst.append(rowlst)
        newlst = []
        for smalllist in inputlst:
            flat_list = list(itertools.chain(*smalllist))
            newlst.append(flat_list)
    return newlst


def apply_lda(inputlst, number, name):
    print("Start LDA...")
    dictionary = corpora.Dictionary(inputlst)
    corpus = [dictionary.doc2bow(text) for text in inputlst]
    ldamodel = gensim.models.LdaMulticore(corpus, num_topics=number, id2word = dictionary, chunksize=100, dtype = np.float32, workers=14)
#     for idx, topic in ldamodel.print_topics(-1):
#     print("Topic: {} \nWords: {}".format(idx, topic))
#     print("\n")  
    topics_list = []
    for document in corpus:
        topics = ldamodel.get_document_topics(document, minimum_probability=0.0)
        onlytopics = [x[1] for x in topics]
        topics_list.append(onlytopics)
    ldadf = pd.DataFrame(topics_list)
    ldadf['author'] = df.index
    ldadf = ldadf.set_index('author')
    columnname = 'lda' + str(number)
    headers = number*[columnname]
    columns = ldadf.columns.values
    arrays = [headers] + [columns]
    ldadf.columns=pd.MultiIndex.from_arrays(arrays)
    return ldadf
   

print("Preprocessing for LDA...")
inputlst = preprocess_lda()
print("LDA with fifty topics: ")
lda50df = apply_lda(inputlst, 50, "ldafifty")
print("LDA with onehundred topics: ")
lda100df = apply_lda(inputlst, 100, "ldahundred")
df = df.join(lda50df, rsuffix="_lda50")
df = df.join(lda100df, rsuffix="_lda100")
df.info(verbose=True)

Preprocessing for LDA...


  0%|          | 0/422 [00:00<?, ?it/s]

LDA with fifty topics: 
Start LDA...
LDA with onehundred topics: 
Start LDA...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 627 columns):
 #    Column                      Dtype  
---   ------                      -----  
 0    (x, index)                  int64  
 1    (x, author)                 object 
 2    (x, complete_body)          object 
 3    (x, doc_body)               object 
 4    (x, probody)                object 
 5    (x, tokens)                 object 
 6    (x, senttokens)             object 
 7    (x, agreeableness)          float64
 8    (x, openness)               float64
 9    (x, conscientiousness)      float64
 10   (x, extraversion)           float64
 11   (x, neuroticism)            float64
 12   (x, agree)                  int64  
 13   (x, openn)                  int64  
 14   (x, consc)                  int64  
 15   (x, extra)                  int64  
 16   (x, neuro)                  int64  
 17   (x, all_utc